In [1]:
import pandas as pd

# 1. 读取并合并数据，纵向连接
df_2015 = pd.read_csv(r'C:\Users\FeatherWounds\Downloads\2015年国内主要城市年度数据.csv')
df_2016 = pd.read_csv(r'C:\Users\FeatherWounds\Downloads\2016年国内主要城市年度数据.csv')
df_2017 = pd.read_csv(r'C:\Users\FeatherWounds\Downloads\2017年国内主要城市年度数据.csv')

# 合并三个数据框
merged_df = pd.concat([df_2015, df_2016, df_2017], axis=0)

In [2]:
# 2. 按照年份来聚合
grouped_by_year = merged_df.groupby('年份')

In [3]:
# 3. 求每年的国内生产总值
gdp_by_year = grouped_by_year['国内生产总值'].sum()
print("每年的国内生产总值:")
print(gdp_by_year)

每年的国内生产总值:
年份
2015    277958.37
2016    300696.10
2017    332562.43
Name: 国内生产总值, dtype: float64


In [4]:
# 4. 处理缺省值，填充为0
filled_df = merged_df.fillna(0)
print("\n处理缺省值后的数据框前几行:")
print(filled_df.head())


处理缺省值后的数据框前几行:
     地区    年份    国内生产总值  第一产业增加值  第二产业增加值   第三产业增加值  社会商品零售总额    货物进出口总额  \
0    北京  2015  23014.59   140.21  4542.64  18331.74   10338.0  319416.16   
1    天津  2015  16538.19   208.82  7704.22   8625.15    5257.3  114282.80   
2   石家庄  2015   5440.60   494.44  2452.40   2493.77    2693.0   12160.29   
3    太原  2015   2735.34    37.40  1020.18   1677.77    1540.8   10677.38   
4  呼和浩特  2015   3090.52   126.23   867.08   2097.21    1353.5    2072.75   

     年末总人口  在岗职工平均工资  普通高等学校在校学生数  医院、卫生院数  房地产开发投资额  
0  1345.20    113073        60.36      701   4177.05  
1  1026.90     81486        51.29      661   1871.55  
2  1028.84     54441        41.98      393    965.13  
3   367.39     60516        42.14      247    597.83  
4   238.58     53698        23.52      174    509.05  


In [5]:
filled_df.to_csv('合并后的城市数据.csv', index=False)